### Q1: Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.You should get a numpy array of size 512.What's the minimal value in this array

In [1]:
from fastembed import TextEmbedding
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [ ]:
documents = "I just discovered the course. Can I join now?"

In [16]:
embedding_model = TextEmbedding(model_handle)
embeddings_generator = embedding_model.embed(documents)
embeddings_list = list(embeddings_generator)
len(embeddings_list[0])

512

In [17]:
import numpy as np
np.min(embeddings_list[0])

np.float64(-0.11726373885183883)

In [18]:
import numpy as np
np.linalg.norm(embeddings_list[0])

np.float64(1.0)

In [19]:
embeddings_list[0].dot(embeddings_list[0])

np.float64(1.0000000000000002)

### Q2 Cosine similarity with another vector: Now let's embed this document: doc = 'Can I still join the course after the start date?'

In [ ]:
query_doc = 'Can I still join the course after the start date?'
query_embeddings_generator = embedding_model.embed(query_doc)
Qembeddings_list = list(query_embeddings_generator)
len(Qembeddings_list[0])

512

In [22]:
embeddings_list[0].dot(Qembeddings_list[0])

np.float64(0.9008528895674548)

## Ranking by cosine

In [ ]:
big_documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [31]:
type(big_documents)

list

In [38]:
texts = [
    f"{doc['text']}"
    for doc in big_documents
]
pprint.pprint(texts)

["Yes, even if you don't register, you're still eligible to submit the "
 'homeworks.\n'
 'Be aware, however, that there will be deadlines for turning in the final '
 "projects. So don't leave everything for the last minute.",
 'Yes, we will keep all the materials after the course finishes, so you can '
 'follow the course at your own pace after it finishes.\n'
 'You can also continue looking at the homeworks and continue preparing for '
 'the next cohort. I guess you can also start working on your final capstone '
 'project.',
 'The purpose of this document is to capture frequently asked technical '
 'questions\n'
 'The exact day and hour of the course will be 15th Jan 2024 at 17h00. The '
 "course will start with the first  “Office Hours'' live.1\n"
 'Subscribe to course public Google Calendar (it works from Desktop only).\n'
 'Register before the course starts using this link.\n'
 'Join the course Telegram channel with announcements.\n'
 "Don’t forget to register in DataTalks.Club's

In [39]:
big_embeddings_generator = embedding_model.embed(texts)
embeddings_big = list(big_embeddings_generator)
len(embeddings_big[0])

512

In [40]:
doc_vectors = np.vstack(embeddings_big)

In [56]:
doc_vectors.shape

(5, 512)

In [57]:
query_doc = 'Can I still join the course after the start date?'
query_vector = np.asarray(next(embedding_model.embed(query_doc)))
query_vector.shape

(512,)

In [ ]:
doc_vectors.dot(query_vector.ravel())

array([0.81060572, 0.84989067, 0.79913355, 0.68130471, 0.67392881])

### Q4: Ranking by cosine, version two full_text = doc['question'] + ' ' + doc['text']

In [61]:
new_texts = [
    f"{doc['question']} {doc['text']}"
    for doc in big_documents
]
pprint.pprint(new_texts)

['Course - Can I still join the course after the start date? Yes, even if you '
 "don't register, you're still eligible to submit the homeworks.\n"
 'Be aware, however, that there will be deadlines for turning in the final '
 "projects. So don't leave everything for the last minute.",
 'Course - Can I follow the course after it finishes? Yes, we will keep all '
 'the materials after the course finishes, so you can follow the course at '
 'your own pace after it finishes.\n'
 'You can also continue looking at the homeworks and continue preparing for '
 'the next cohort. I guess you can also start working on your final capstone '
 'project.',
 'Course - When will the course start? The purpose of this document is to '
 'capture frequently asked technical questions\n'
 'The exact day and hour of the course will be 15th Jan 2024 at 17h00. The '
 "course will start with the first  “Office Hours'' live.1\n"
 'Subscribe to course public Google Calendar (it works from Desktop only).\n'
 'Regist

In [62]:
big_embeddings_generator = embedding_model.embed(new_texts)
embeddings_big = list(big_embeddings_generator)
doc_vectors = np.vstack(embeddings_big)

In [63]:
doc_vectors.dot(query_vector.ravel())

array([0.92732523, 0.85948475, 0.83966021, 0.76320298, 0.76044493])

In [64]:
## Selecting the embedding model

In [ ]:
TextEmbedding.list_supported_models()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [71]:
import json

EMBEDDING_DIMENSIONALITY = 384

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-small-en-v1.5-onnx-q",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.067,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "snowflake/snowflake-arctic-embed-xs",
  "sou

## Q6: Indexing with Qdrant

In [72]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [78]:
documents[:3]

[{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
  'section': 'General course-related questions',
  'question': 'How do I sign up?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
  'section': 'General course-related questions',
  'question': 'Is it going to be live? When?',
  'course': 'machi

In [75]:
model_handle = "BAAI/bge-small-en"
EMBEDDING_DIMENSIONALITY = 384

In [74]:
from qdrant_client import QdrantClient, models
client = QdrantClient("http://localhost:6333")

In [77]:
# Define the collection name
collection_name = "zoomcamp-hw2"
client.delete_collection(collection_name=collection_name)

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [79]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [80]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [81]:
client.upsert(collection_name=collection_name, points=points)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [82]:
q1 = "I just discovered the course. Can I join now?"


In [83]:
course = 'machine-learning-zoomcamp'
query_points = client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=q1,
        model=model_handle 
    ),
    query_filter=models.Filter( 
        must=[
            models.FieldCondition(
                key="course",
                match=models.MatchValue(value=course)
            )
        ]
    ),
    limit=5,
    with_payload=True
)
results = []
for point in query_points.points:
    results.append(point.payload)


In [86]:
pprint.pprint(query_points)

QueryResponse(points=[ScoredPoint(id=14, version=2, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=6, version=2, score=0.86918855, payload={'text': 'Approximately 4 months, but may take more if you want to do some extra activities (an extra project, an article, etc)', 'section': 'General course-related questions', 'question': 'How long is the course?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=